<a href="https://colab.research.google.com/github/bagascahyawiguna/DicodingBMLP/blob/main/%5BKlasifikasi%5D_Submission_Akhir_BMLP_Bagas_Cahyawiguna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import Library**

Pada tahap ini, Anda perlu mengimpor beberapa pustaka (library) Python yang dibutuhkan untuk analisis data dan pembangunan model machine learning.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# **2. Memuat Dataset dari Hasil Clustering**

Memuat dataset hasil clustering dari file CSV ke dalam variabel DataFrame.

In [3]:
transactions_df_cluster = pd.read_csv('/content/drive/MyDrive/Dataset/hasil_clustering.csv')
transactions_df_cluster.head()

,AccountID,TransactionAmount,Location,DeviceID,IP Address,MerchantID,CustomerAge,TransactionDuration,LoginAttempts,AccountBalance,...,Month,DaysSinceLastTransaction,TransactionType_Debit,Channel_Branch,Channel_Online,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student,Cluster,Cluster_PCA
0,126,-0.971275,36,365,186,14,1.423718,-0.552443,-0.206794,-0.000537,...,4,-573,True,False,False,False,False,False,0,0
1,450,0.269440,15,50,82,51,1.311287,0.305314,-0.206794,2.216472,...,6,-496,True,False,False,False,False,False,0,0
2,18,-0.586882,23,229,343,8,-1.443277,-0.909842,-0.206794,-1.023534,...,7,-483,True,False,True,False,False,True,0,0
3,68,-0.387456,33,182,300,1,-1.049768,-1.353017,-0.206794,0.885797,...,5,-549,True,False,True,False,False,True,0,0
4,406,-0.973468,1,298,501,90,-1.049768,1.120184,-0.206794,0.593589,...,10,-385,False,False,True,False,False,True,-1,0


In [4]:
transactions_df_cluster.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2512 entries, 0 to 2511
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   AccountID                    2512 non-null   int64  
 1   TransactionAmount            2512 non-null   float64
 2   Location                     2512 non-null   int64  
 3   DeviceID                     2512 non-null   int64  
 4   IP Address                   2512 non-null   int64  
 5   MerchantID                   2512 non-null   int64  
 6   CustomerAge                  2512 non-null   float64
 7   TransactionDuration          2512 non-null   float64
 8   LoginAttempts                2512 non-null   float64
 9   AccountBalance               2512 non-null   float64
 10  DayOfWeek                    2512 non-null   int64  
 11  Hour                         2512 non-null   int64  
 12  Month                        2512 non-null   int64  
 13  DaysSinceLastTrans

In [5]:
# Cek missing values
print("\nMissing values per fitur:")
print(transactions_df_cluster.isnull().sum())


Missing values per fitur:
AccountID                      0
TransactionAmount              0
Location                       0
DeviceID                       0
IP Address                     0
MerchantID                     0
CustomerAge                    0
TransactionDuration            0
LoginAttempts                  0
AccountBalance                 0
DayOfWeek                      0
Hour                           0
Month                          0
DaysSinceLastTransaction       0
TransactionType_Debit          0
Channel_Branch                 0
Channel_Online                 0
CustomerOccupation_Engineer    0
CustomerOccupation_Retired     0
CustomerOccupation_Student     0
Cluster                        0
Cluster_PCA                    0
dtype: int64


# **3. Data Splitting**

Tahap Data Splitting bertujuan untuk memisahkan dataset menjadi dua bagian: data latih (training set) dan data uji (test set).

In [6]:
from sklearn.model_selection import train_test_split

# Memisahkan fitur (X) dan target (y)
X = transactions_df_cluster[['TransactionAmount', 'CustomerAge', 'TransactionDuration', 'LoginAttempts', 'AccountBalance', 'TransactionType_Debit']]
y = transactions_df_cluster['Cluster_PCA']  # Label cluster sebagai target

# Split data menjadi set pelatihan dan set uji
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Ubah label kelas
y_train = np.where(y_train == -1, 0, y_train + 1)  # Ubah -1 menjadi 0, dan tambahkan 1 ke label lainnya
y_test = np.where(y_test == -1, 0, y_test + 1)    # Lakukan hal yang sama untuk y_test

# Tampilkan bentuk set pelatihan dan set uji untuk memastikan split
print(f"Training set shape: X_train={X_train.shape}, y_train={y_train.shape}")
print(f"Test set shape: X_test={X_test.shape}, y_test={y_test.shape}")

Training set shape: X_train=(2009, 6), y_train=(2009,)
Test set shape: X_test=(503, 6), y_test=(503,)


# **4. Membangun Model Klasifikasi**


## **a. Membangun Model Klasifikasi**

Setelah memilih algoritma klasifikasi yang sesuai, langkah selanjutnya adalah melatih model menggunakan data latih.

Berikut adalah rekomendasi tahapannya.
1. Pilih algoritma klasifikasi yang sesuai, seperti Logistic Regression, Decision Tree, Random Forest, atau K-Nearest Neighbors (KNN).
2. Latih model menggunakan data latih.

### 1. Menggunakan Algoritma Random Forest

In [20]:
from sklearn.ensemble import RandomForestClassifier

# Inisialisasi model
model_rf = RandomForestClassifier(random_state=42, class_weight='balanced')  # Parameter class_weight untuk menangani imbalance class

# Latih model menggunakan training set
model_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', random_state=42)

### 2. Menggunakan algoritma XGBoost

In [23]:
from xgboost import XGBClassifier

# Inisialisasi model XGBoost
model_xgb = XGBClassifier(random_state=42, scale_pos_weight=(len(y_train) / sum(y_train == 1)  # Menangani imbalance class
))

# Latih model XGBoost
model_xgb.fit(X_train, y_train)

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:39:34] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

Saya menggunakan algoritma Random Forest dengan alasan:
- Random Forest adalah metode ensemble learning yang robust terhadap overfitting dan dapat menangani data tidak seimbang dengan baik.
- Dapat menangani hubungan non-linear antara fitur dan target.
- Menyediakan feature importance, yang membantu memahami fitur mana yang paling berpengaruh dalam mendeteksi fraud.

Saya mencoba menggunakan algoritma lain yaitu XGBoost untuk membandingkannya dengan algoritma Random Forest dengan alasan algoritma boosting merupakan algoritma yang powerful untuk dataset tidak seimbang, dalam kasus saya dataset memiliki outlier yang diasumsikan sebagai anomali karena merupakan minoritas data dari sebagian besar data.

## **b. Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Lakukan prediksi menggunakan data uji.
2. Hitung metrik evaluasi seperti Accuracy dan F1-Score (Opsional: Precision dan Recall).
3. Buat confusion matrix untuk melihat detail prediksi benar dan salah.

### Menggunakan Random Forest

In [17]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix

# Prediksi menggunakan test set
y_pred = model_rf.predict(X_test)

# Hitung metrik evaluasi
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("F1-Score:", f1)
print("Precision:", precision)
print("Recall:", recall)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.9900596421471173
F1-Score: 0.9851142892892396
Precision: 0.980218095008478
Recall: 0.9900596421471173
Confusion Matrix:
 [[  0   2   0   0]
 [  0 498   0   0]
 [  0   1   0   0]
 [  0   2   0   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


### Menggunakan XGBoost

In [18]:
# Prediksi menggunakan model XGBoost
y_pred_xgb = model_xgb.predict(X_test)

# Hitung metrik evaluasi untuk XGBoost
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
precision_xgb = precision_score(y_test, y_pred_xgb, average='weighted')
recall_xgb = recall_score(y_test, y_pred_xgb, average='weighted')

# Tampilkan metrik evaluasi XGBoost
print("Accuracy (XGBoost):", accuracy_xgb)
print("F1-Score (XGBoost):", f1_xgb)
print("Precision (XGBoost):", precision_xgb)
print("Recall (XGBoost):", recall_xgb)

# Confusion Matrix untuk XGBoost
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
print("Confusion Matrix (XGBoost):\n", conf_matrix_xgb)

Accuracy (XGBoost): 0.9920477137176938
F1-Score (XGBoost): 0.9887501656726309
Precision (XGBoost): 0.986146863836899
Recall (XGBoost): 0.9920477137176938
Confusion Matrix (XGBoost):
 [[  1   1   0   0]
 [  0 498   0   0]
 [  0   1   0   0]
 [  0   2   0   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Analisis:
- XGBoost sedikit lebih baik daripada Random Forest di semua metrik evaluasi, dengan peningkatan tipis pada accuracy, F1-score, precision, dan recall.
- Perbedaan utama pada confusion matrix: XGBoost berhasil memprediksi 1 sampel dari cluster 0 dengan benar, sementara Random Forest tidak berhasil memprediksi cluster 0 sama sekali.
- Asumsi saya, cluster 2 dan 3 mungkin berisi noise atau outlier yang jumlahnya sangat sedikit dibandingkan dengan data dominan sehingga kedua algoritma tersebut tidak dapat memprediksi dengan baik pada kedua cluster tersebut.

## **c. Tuning Model Klasifikasi (Optional)**

Gunakan GridSearchCV, RandomizedSearchCV, atau metode lainnya untuk mencari kombinasi hyperparameter terbaik

### Tuning model klasifikasi Random Forest

In [19]:
from sklearn.model_selection import GridSearchCV

# Definisikan parameter grid
param_grid = {
    'n_estimators': [100, 200, 300],  # Jumlah tree dalam forest
    'max_depth': [None, 10, 20, 30],  # Kedalaman maksimum setiap tree
    'min_samples_split': [2, 5, 10],  # Jumlah minimum sampel yang diperlukan untuk memisahkan node
    'min_samples_leaf': [1, 2, 4],    # Jumlah minimum sampel di leaf node
    'class_weight': ['balanced', None]  # Menangani imbalance class
}

# Inisialisasi model
model_rf = RandomForestClassifier(random_state=42)

# Gunakan GridSearchCV
grid_search = GridSearchCV(
    estimator=model_rf,
    param_grid=param_grid,
    scoring='f1_weighted',  # Gunakan F1-score sebagai metrik evaluasi
    cv=5,                  # 5-fold cross-validation
    n_jobs=-1,             # Gunakan semua core CPU
    verbose=2              # Tampilkan progress
)

# Latih GridSearchCV pada data training
grid_search.fit(X_train, y_train)

# Hasil hyperparameter terbaik
print("Hyperparameter Terbaik:", grid_search.best_params_)

# Model terbaik setelah tuning
best_model = grid_search.best_estimator_

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Hyperparameter Terbaik: {'class_weight': 'balanced', 'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 300}


### Tuning model klasifikasi XGBoost

In [22]:
from sklearn.model_selection import GridSearchCV

# Definisikan parameter grid untuk XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

# Gunakan GridSearchCV untuk tuning XGBoost
grid_search_xgb = GridSearchCV(
    estimator=model_xgb,
    param_grid=param_grid_xgb,
    scoring='f1_weighted',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Latih GridSearchCV pada data training
grid_search_xgb.fit(X_train, y_train)

# Hasil hyperparameter terbaik
print("Hyperparameter Terbaik (XGBoost):", grid_search_xgb.best_params_)

# Model terbaik setelah tuning
best_model_xgb = grid_search_xgb.best_estimator_

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Hyperparameter Terbaik (XGBoost): {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [06:39:20] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


## **d. Evaluasi Model Klasifikasi setelah Tuning (Optional)**

Berikut adalah rekomendasi tahapannya.
1. Gunakan model dengan hyperparameter terbaik.
2. Hitung ulang metrik evaluasi untuk melihat apakah ada peningkatan performa.

### Random Forest

In [21]:
# Prediksi menggunakan model terbaik
y_pred_tuned = best_model.predict(X_test)

# Hitung metrik evaluasi setelah tuning
accuracy_tuned = accuracy_score(y_test, y_pred_tuned)
f1_tuned = f1_score(y_test, y_pred_tuned, average='weighted')
precision_tuned = precision_score(y_test, y_pred_tuned, average='weighted')
recall_tuned = recall_score(y_test, y_pred_tuned, average='weighted')

# Tampilkan metrik evaluasi setelah tuning
print("Hasil Evaluasi Setelah Tuning: ")
print("Accuracy setelah Tuning:", accuracy_tuned)
print("F1-Score setelah Tuning:", f1_tuned)
print("Precision setelah Tuning:", precision_tuned)
print("Recall setelah Tuning:", recall_tuned)

# Confusion Matrix setelah Tuning
conf_matrix_tuned = confusion_matrix(y_test, y_pred_tuned)
print("Confusion Matrix setelah Tuning:\n", conf_matrix_tuned)

# Bandingkan dengan Performa Sebelum Tuning
y_pred = model_rf.predict(X_test)

# Hitung metrik evaluasi sebelum tuning
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')

# Tampilkan perbandingan metrik evaluasi
print("\nPerbandingan Performa Sebelum dan Setelah Tuning:")
print(f"Accuracy: Sebelum = {accuracy}, Setelah = {accuracy_tuned}")
print(f"F1-Score: Sebelum = {f1}, Setelah = {f1_tuned}")
print(f"Precision: Sebelum = {precision}, Setelah = {precision_tuned}")
print(f"Recall: Sebelum = {recall}, Setelah = {recall_tuned}")

# Confusion Matrix Sebelum Tuning
conf_matrix = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix Sebelum Tuning:\n", conf_matrix)


Hasil Evaluasi Setelah Tuning: 
Accuracy setelah Tuning: 0.9900596421471173
F1-Score setelah Tuning: 0.9877551507770196
Precision setelah Tuning: 0.9861311174866966
Recall setelah Tuning: 0.9900596421471173
Confusion Matrix setelah Tuning:
 [[  0   2   0   0]
 [  1 497   0   0]
 [  0   1   0   0]
 [  0   1   0   1]]

Perbandingan Performa Sebelum dan Setelah Tuning:
Accuracy: Sebelum = 0.9900596421471173, Setelah = 0.9900596421471173
F1-Score: Sebelum = 0.9851142892892396, Setelah = 0.9877551507770196
Precision: Sebelum = 0.980218095008478, Setelah = 0.9861311174866966
Recall: Sebelum = 0.9900596421471173, Setelah = 0.9900596421471173

Confusion Matrix Sebelum Tuning:
 [[  0   2   0   0]
 [  0 498   0   0]
 [  0   1   0   0]
 [  0   2   0   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Analisis:
- Accuracy: Tetap sama (0.9901)
- F1-Score: Meningkat dari 0.9851 menjadi 0.9878 (+0.0027)
- Precision: Meningkat dari 0.9802 menjadi 0.9861 (+0.0059)
- Recall: Tetap sama (0.9901)
Sebelum tuning, model tidak bisa memprediksi kelas 0 dan kelas 2 dan Semua prediksi masuk ke kelas 1 (498 benar, 5 salah). Sedangkan sesudah tuning model mulai bisa memprediksi kelas 3 (1 prediksi benar) dan total kesalahan klasifikasi berkurang dari 5 menjadi 4 item
Distribusi prediksi lebih baik (ada prediksi di kelas 3)

### XGBoost

In [25]:
# Prediksi menggunakan model terbaik (XGBoost setelah tuning)
y_pred_xgb_tuned = best_model_xgb.predict(X_test)

# Hitung metrik evaluasi setelah tuning
accuracy_xgb_tuned = accuracy_score(y_test, y_pred_xgb_tuned)
f1_xgb_tuned = f1_score(y_test, y_pred_xgb_tuned, average='weighted')
precision_xgb_tuned = precision_score(y_test, y_pred_xgb_tuned, average='weighted')
recall_xgb_tuned = recall_score(y_test, y_pred_xgb_tuned, average='weighted')

# Tampilkan metrik evaluasi setelah tuning
print("Hasil Evaluasi XGBoost Setelah Tuning: ")
print("Accuracy setelah Tuning (XGBoost):", accuracy_xgb_tuned)
print("F1-Score setelah Tuning (XGBoost):", f1_xgb_tuned)
print("Precision setelah Tuning (XGBoost):", precision_xgb_tuned)
print("Recall setelah Tuning (XGBoost):", recall_xgb_tuned)

# Confusion Matrix setelah Tuning
conf_matrix_xgb_tuned = confusion_matrix(y_test, y_pred_xgb_tuned)
print("Confusion Matrix setelah Tuning (XGBoost):\n", conf_matrix_xgb_tuned)


Hasil Evaluasi XGBoost Setelah Tuning: 
Accuracy setelah Tuning (XGBoost): 0.9920477137176938
F1-Score setelah Tuning (XGBoost): 0.9887501656726309
Precision setelah Tuning (XGBoost): 0.986146863836899
Recall setelah Tuning (XGBoost): 0.9920477137176938
Confusion Matrix setelah Tuning (XGBoost):
 [[  1   1   0   0]
 [  0 498   0   0]
 [  0   1   0   0]
 [  0   2   0   0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Analisis:
Terlihat tidak ada perbedaan antara sebelum dan setelah tuning

## **e. Analisis Hasil Evaluasi Model Klasifikasi**

Berikut adalah **rekomendasi** tahapannya.
1. Bandingkan hasil evaluasi sebelum dan setelah tuning (jika dilakukan).
  - Random Forest
    - Sebelum Tuning
      - Accuracy: 0.9901
      - F1-Score: 0.9851
      - Precision: 0.9802
      - Recall: 0.9901
      - Confusion Matrix: Model hanya memprediksi kelas 1, tidak dapat memprediksi kelas 0, 2, dan 3
    - Setelah Tuning
      - Accuracy: 0.9901 (tidak berubah)
      - F1-Score: 0.9878 (meningkat +0.0027)
      - Precision: 0.9861 (meningkat +0.0059)
      - Recall: 0.9901 (tidak berubah)
      - Confusion Matrix: Model berhasil memprediksi 1 instance kelas 3 dengan benar, namun masih tidak dapat memprediksi kelas 0 dan 2
  - XGBoost
    - Nilai metrik dan confusion matrix identik, menunjukkan parameter tuning tidak memberikan perubahan
    - Accuracy: 0.9920
    - F1-Score: 0.9888
    - Precision: 0.9861
    - Recall: 0.9920
    - Confusion matrix menunjukkan model dapat memprediksi 1 instance kelas 0 dengan benar, namun tidak dapat memprediksi kelas 2 dan 3
2. Identifikasi kelemahan model, seperti:
  - Random Forest:
    - Model memiliki recall 0 untuk kelas 0, 2, dan 3 sebelum tuning, dan recall 0 untuk kelas 0 dan 2 setelah tuning sehingga Precision dan Recall rendah untuk kelas tertentu
    - Model cenderung mengklasifikasikan semua data sebagai kelas 1 (kelas mayoritas)
    - Terlihat sepertinya dari confusion matrix bahwa data didominasi oleh kelas 1 (498 sampel) sementara kelas lainnya sangat sedikit
  - XGBoost
    - Model memiliki recall 0 untuk kelas 2 dan 3
    - Sama halnya seperti pada Random Forest, model mengalami kesulitan mengklasifikasikan kelas minoritas
    - Parameter tuning sepertinya tidak memberikan peningkatan performa, menunjukkan model mungkin sudah optimal atau mungkin selanjutnya perlu pendekatan tuning yang berbeda
  - Kelemahan Umum
    - Kedua model mengalami kesulitan dalam memprediksi kelas minoritas (kelas 0, 2, dan 3)
    - Meskipun akurasi keseluruhan terlihat tinggi (>99%), hal ini mungkin bisa tidak efektif karena ketidakseimbangan kelas yang ekstrim
    - Sepertinya tidak ada tanda overfitting yang jelas, namun ada kemungkinan underfitting pada kelas minoritas karena jumlah sampel yang terlalu sedikit
3. Berikan rekomendasi tindakan lanjutan, seperti mengumpulkan data tambahan atau mencoba algoritma lain jika hasil belum memuaskan.
  - Mungkin selanjutnya saya perlu menyesuaikan parameter tuning dengan fokus pada peningkatan performa kelas minoritas
  - Perlu mencoba menggunakan metrik evaluasi yang lebih sensitif terhadap ketidakseimbangan kelas, seperti Balanced Accuracy atau F2-Score

Secara keseluruhan, XGBoost sedikit lebih unggul dalam metrik performa umum. Namun, setelah melakukan Tuning model Random Forest menunjukkan kemampuan lebih baik dalam memprediksi kelas 3.